<a href="https://colab.research.google.com/github/zhuan0122/Machine_Learing_Handson_Python_in_Data_science/blob/master/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

### Importing the libraries

In [ ]:
# import the library for processing the images and training NNs 
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!python3 --version

Python 3.6.9


In [ ]:
tf.__version__

'2.3.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# we need to transform the trainning data to aviod overfitting which means a high accuracy on traninning data set but a relative
# accuracy on the test data. It is caused by the lack of the variaty or diversity of the images. so Do transformation includes 
# geometrical transformation, zoom in and out, horitontal flips and so on which is also called Image augmentation 
# the tool for transfomation is from keras API. search keras API in google 
# we can be free to try any other  transformation method to see if the accuaracy is increased or not but here we just copy 
# the given codes from the API page 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# apply the transformation to the given data set and from the experience we usually downsize the image pixel to be 64x64
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary') # we take a binary catogery here 


### Preprocessing the Test set

In [ ]:
# test image we need to keep it as original without any transformation but with the same scaling 
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
# initize the input layer as a sequence of neurons. the total number of neuron would be decided automatically by the input 
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
# the first convolution layers is got by convulzed. the filter is the feature detector matrix, we only give how many filters we would use
# in this layer to get the feature maps. each filter would filter or detect different feature. and the filter matrix size 3 means is 3x3 matrix
# all activation function except the out put layer would be recitifier. Input_shape is the image shape. Colored images are RGB shape 
# which is [0~255, 0~255,3], for simple computation, we use 64x64x3. if it is black or white images. then it is [, , 1]
# all images are 2D dimentional. 
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [ ]:
# pooling here is max pooling. which means takes the maximum value of each pool. pool size is the 2x2 matrix and stride is 2
# see the notebook for details of understanding what does these mean
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
# exactly the same as above but no needs for clarify the input shape of the image here and after convolution, it will follow with 
# the pooling 
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())
# flattening the final result of the last pooling and make the data as the sequece of vector 

### Step 4 - Full Connection

In [ ]:
# dense method here is used to full connect the nerous of this layer to the last layer
# this fully connected layer is the first hidden layer of the ANN after the flattening layer which literaly is the input layer of the ANN
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))


In [ ]:
# I want to try add second hidden layer here to see if the accuracy is enhanced or not
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
# adam is the optimizer to perform stochastic gradien descent to find the lowest cost function and update the weights and filter matrix
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# the difference fit here from the ANNs is we also evaluate the NNS on the test test. epochs is the rounds we gonna run NNS on the all 
# data set, it could be tunned 
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np 
from keras.preprocessing import image
# load the sigle image
test_image=image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
#we need to transfer the input image to the array because the 'predict' method only work with the array as input. not directly 2D image
test_image = image.img_to_array(test_image)
# and also the NNS are trainning with 32 batch size whihc means each time, NNS will be fed with 32 images as a batch, since we 
# have 8000 total, so it will has 250 steps for all data finished running. and then we will repeat this 'epoch' times
# we need to add a fake dimension for the batch 32 even only one sigle image in this batch but we still need to do this for 
# the same input to the tarinning set 
test_image = np.expand_dims(test_image, axis = 0) # 0 means rows  is the first dimension.
# first dimension means when we access the image we need to call the first dimention first to get into the first batch and then call the 
# specific image. whcih also could understand as columns and rows. rows are image list number, and each 32 rows are one bacth and columns 
# is the images object 
result = cnn.predict(test_image)
# from this contribute we could see 1 is dog or cat.
training_set.class_indices
# log result
if result [0][0] == 1:# first 0 is the batch dimension, means first batch and second zero means the first image predict result
  prediction == 'dog'
else:
  prediction=='cat'




In [ ]:
print(prediction)